# OS - ELM

## Importing Libraries

In [ ]:
import torch
import time
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

## MEFAR Dataset Class

In [ ]:
class MEFAR_Dataset(Dataset):
    def __init__(self, features, labels):

        scaler = StandardScaler()

        self.features = torch.tensor(scaler.fit_transform(features), dtype = torch.float32)
        self.labels = torch.tensor(scaler.fit_transform(labels), dtype = torch.float32)

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]


## OS - ELM Class

In [ ]:
class OSELM(nn.Module):
    def __init__(self, n_hidden_neurons, device)
